# From stand-alone python to SML
the pourpose of this experiment is to re-write the 
existing code for this kpi about devices and battery in our sml language.
In this notebook we will review the current code starting with data from an existing json file with the filtered data as it will be provided by the framework.

In [97]:
import json
with open('2106_08_22_10_18_json.json') as json_data:
    d = json.load(json_data)

In [98]:
import pandas as pd 
data = pd.DataFrame(d)

In [66]:
#cleaning non numerical data
cols = data.select_dtypes(['object'])
data[cols.columns] = cols.apply(lambda x: x.str.strip())

In [103]:
# at this point we have a data we want. But, the dates are string 
# and cannot be compared
data.dtypes

StatType     int64
_id         object
intvalue     int64
dtype: object

In [6]:
# this is something we could have avoided by providing 
# the data in the right way from the beginning
data['time_stamp'] = pd.to_datetime(data['time_stamp'], format='%Y-%m-%d %H:%M:%S')

In [7]:
data.dtypes

StatType               int64
_id                   object
devid                 object
intvalue               int64
time_stamp    datetime64[ns]
dtype: object

In [68]:
#SORTING ...THIS SEEMS SOMEHOW IMPORTNANT
data.set_index(['devid', 'time_stamp'], inplace=True)
data.sort_index(level=1, inplace=True)

In [70]:
dischargedGroup = (data.groupby(level=0, sort=False)['intvalue']
                           .apply(list))

In [53]:
code = "x+1"

In [54]:
code

'x+1'

In [55]:
x=1
eval(code)

2

In [110]:
code="""
def f(data):
                cols = data.select_dtypes(['object'])
                data[cols.columns] = cols.apply(lambda x: x.str.strip())
                data['time_stamp'] = pd.to_datetime(data['time_stamp'], format='%Y-%m-%d %H:%M:%S')
                data.set_index(['devid', 'time_stamp'], inplace=True) 
                data.sort_index(level=1, inplace=True)
                dischargedGroup = (data.groupby(level=0, sort=False)['intvalue'].apply(list))
                def check(line): 
                    oldval = 100
                    for i in line:
                        if (i > oldval) | (i < threshold):
                            return 1
                            break
                        else: 
                            oldval = i
                        return 0
                discharged = dischargedGroup.apply(check)
"""

In [114]:
import json
with open('2106_08_22_10_18_json.json') as json_data:
    d = json.load(json_data)
import pandas as pd 
data = pd.DataFrame(d)
start = ''
shift = 0
threshold = 10
exec(code)
f(data)
print(data)

                                                       StatType  \
devid                             time_stamp                      
73E65B76064901080007-21C0145F2600 2016-08-22 10:00:00        -1   
73E65B76064901080101-1E10D47A2600 2016-08-22 10:00:00        -1   
73E65B76064901080104-1FA0CC110600 2016-08-22 10:00:00        -1   
73E65B76064901080114-169030060600 2016-08-22 10:00:00        -1   
73E65B7606490108000D-1BB066730600 2016-08-22 10:19:59        -1   
73E65B76064901080001-0C2024170600 2016-08-22 10:29:59        -1   
73E65B76064901080001-1860F2240600 2016-08-22 10:29:59        -1   
73E65B76064901080001-1B8048550600 2016-08-22 10:29:59        -1   
73E65B76064901080002-1B9096690600 2016-08-22 10:29:59        -1   
73E65B76064901080003-1870AA3B0600 2016-08-22 10:29:59        -1   
73E65B76064901080004-0670E06A2600 2016-08-22 10:29:59        -1   
73E65B76064901080004-0EC008320600 2016-08-22 10:29:59        -1   
73E65B76064901080004-1A601C6F0600 2016-08-22 10:29:59        -

# Example of how to execute python code stored in strings
import sys
import StringIO

### create file-like string to capture output
codeOut = StringIO.StringIO()
codeErr = StringIO.StringIO()

code = """
def f(x):
    x = x + 1
    return x

print 'This is my output.'
"""

### capture output and errors
sys.stdout = codeOut
sys.stderr = codeErr

exec code

### restore stdout and stderr
sys.stdout = sys.__stdout__
sys.stderr = sys.__stderr__

print f(4)

s = codeErr.getvalue()

print "error:\n%s\n" % s

s = codeOut.getvalue()

print "output:\n%s" % s

codeOut.close()
codeErr.close()
